# Answering Business Questions of a Digital Store using SQL

## Introduction

In this project, we will use the `Chinook` database, which represents a digital media store, including tables for artists, albums, media tracks, invoices and customers.

Below is a schema of the database.

![](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)

### Connecting and exploring the database

We will start by connecting our Jupyter Notebook to the database file.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

Now we will get the list of all the tables and views in our database.

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


We have 11 tables that are related as shown in the image showed before. Let's get familiar with our data by inspecting some of the tables.

In [3]:
%%sql
SELECT * FROM invoice
LIMIT 5;

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83


In [4]:
%%sql
SELECT * FROM employee
LIMIT 5;

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


## Selecting albums to include in the media store

The Chinook media store has just signed a deal with a new record label, and we have been tasked with **selecting the first 3 albums that will be added to the store**, from a list of 4. 

All four albums are by artists that do not have any tracks in the store right now - we have the artist names, and the genre of music they produce:

| Artist Name     | Genre |
| :----------- | :----------- |
| Regal      | Hip-Hop       |
| Red Tone | Punk      |
| Meteor and the Girls | Pop     |
| Slim Jim Bites | Blues      |

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA.

Based on the information that we have, we will analyse **which genres sell the best in the USA** to choose the 3 albums that should be purchased for the store. 

Let's write a query that returns each genre with the number of tracks sold in the USA, in absolute numbers and in percentages.

We are going to use the following tables:
- `genre` to get the genre name
- `track` to link each track to its genre
- `customer` to filter by country
- `invoice` and `invoice_line` to get the number of track sold and link the other two tables.

In [5]:
%%sql

WITH
    sales_by_genre AS
        (
        SELECT
            g.name AS genre
            , COUNT(il.track_id) AS sold_tracks
        FROM invoice i
        INNER JOIN customer c 
            ON c.customer_id = i.customer_id
        INNER JOIN invoice_line il
            ON il.invoice_id = i.invoice_id
        INNER JOIN track t
            ON t.track_id = il.track_id
        INNER JOIN genre g
            ON g.genre_id = t.genre_id
        WHERE c.country = "USA"
        GROUP BY g.genre_id
        )

SELECT
    genre
    , sold_tracks
    , CAST(sold_tracks AS FLOAT) /
        (SELECT SUM(sold_tracks)
         FROM sales_by_genre)*100 AS percentage_sold_tracks
FROM sales_by_genre
GROUP BY genre
ORDER BY 2 DESC
LIMIT 10;

Done.


genre,sold_tracks,percentage_sold_tracks
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.425309229305423
Alternative,35,3.3301617507136063
Latin,22,2.093244529019981
Pop,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


Looking at the results, the genres of the 4 proposed albums appear among the top 10 genres with more tracks sold in the USA. Howerver, the most popular are Punk, Blues and Pop in this order. Thus, we recommend to puchase the following albums:

- Red Tone (Punk)
- Meteor and the Girls (Pop)
- Slim Jim Bites (Blues)

## Analysing Employee Sales Perfomance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. 

We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

To find it out, we will write a query that fins the total dollar amount of sales assigned to each sales support agent within the company. We will also take into account the hiring date for each imployee to see the average total sales per month.

We will use the following tables from the database:

- `employee` to get the employee information and filter by sales support agents (first name, last name, title, hiring date)
- `customer` to link the employee with each purchase transction
- `invoice` to get dollar amount of each purchase

In [6]:
%%sql

WITH employee_sales AS
    (
    SELECT
        c.support_rep_id AS employee_id
        , SUM(i.total) AS total_sales
    FROM customer c
    INNER JOIN invoice i
        ON i.customer_id = c.customer_id
    GROUP BY 1
    )

SELECT
    e.first_name || " " || e.last_name AS employee_name
    , es.total_sales
    , e.hire_date
FROM employee e
LEFT JOIN employee_sales es
    ON es.employee_id = e.employee_id
WHERE e.title = "Sales Support Agent"
ORDER BY es.total_sales DESC;

Done.


employee_name,total_sales,hire_date
Jane Peacock,1731.510000000004,2017-04-01 00:00:00
Margaret Park,1584.0000000000032,2017-05-03 00:00:00
Steve Johnson,1393.9200000000028,2017-10-17 00:00:00


Although Steve Johnson is the employee with lower sales (1393.92$), he has been in the company for about 5-6 months less than the two other sales support agents.

The difference in sales between Jane (the top employee) and Steve (the bottom employee), roughly corresponds with the differences in their hiring dates (24% vs 17% difference), knowing that the date of the last invoice is 2020-12-30.

## Analysing Sales by Country

Our next task is to analyze the sales data for customers from each different country. We have been given guidance to use the country value from the customers table, rather than the billing address in the invoice table.

We have been directed to calculate the following data for each country:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Because there are a number of countries with only one customer, we will group these customers as "Other" in 
our analysis.

The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.

In [7]:
%%sql

WITH
    country_customer_sales AS
        (
        SELECT
            CASE
                WHEN (SELECT COUNT(*)
                      FROM customer
                      WHERE country = c.country
                     ) = 1 THEN "Other"
                ELSE c.country
                END AS country
            , c.customer_id
            , i.invoice_id
            , i.total AS invoice_value
        FROM customer c  
        INNER JOIN invoice i ON i.customer_id = c.customer_id
        ),
    country_sales AS
        (
        SELECT
            country
            , COUNT(DISTINCT customer_id) AS total_customers
            , SUM(invoice_value) AS total_sales
            , SUM(invoice_value)/ COUNT(DISTINCT customer_id) AS avg_sales_customer
            , SUM(invoice_value)/ COUNT(DISTINCT invoice_id) AS avg_order
            , CASE
                WHEN country = "Other" THEN 1
                ELSE 0
                END AS sort
        FROM country_customer_sales
        GROUP BY country
        ORDER BY sort, total_sales DESC
        )

SELECT
    country
    , total_customers
    , total_sales
    , avg_sales_customer
    , avg_order
FROM country_sales

Done.


country,total_customers,total_sales,avg_sales_customer,avg_order
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
United Kingdom,3,245.52,81.84,8.768571428571429
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Other,15,1094.9400000000003,72.99600000000002,7.44857142857143


We see that the USA is the country with highest total sales, followed by Canada.

Czech Republic has the customers with higher lifetime value, followed by Portugal and India. However, the sample is very small, so the data may not be representative of the general customer behaviour.

I would recommend investing in these countries with small marketing campaigns. Once we have more sales data, we can decide better if a bigger investment is worth it.

## Album vs Individual Tracks

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

- purchase a whole album
- purchase a collection of one or more individual tracks

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). Tracks in album or purchased separately are chargerd the same price.

Management are currently considering changing their purchasing strategy to save money - purchase only the most popular tracks from each album from record companies, instead of the whole album.

We have been asked to find out **what percentage of purchases are individual tracks vs whole albums**, so that management can understand the effect this decision might have on overall revenue.

We have two edge cases to consider:

- Albums that have only one or two tracks are likely to be purchased by customers as individual tracks.
- Customers may decide to manually select every track from an album to purchase it.

In the first case, since it will not make a big difference on revenue, we can safely ignore it. In the second case, the company has previously done analysis to confirm that it does not happen often, so we can ignore as well.

To answer the question, we're going to identify whether each invoice has all the tracks from an album. 

In [8]:
%%sql

DROP VIEW IF EXISTS invoice_tracks;
DROP VIEW IF EXISTS album_tracks;

CREATE VIEW invoice_tracks AS
    SELECT
        il.invoice_id
        , il.track_id
        , t.album_id
    FROM invoice_line il
    INNER JOIN track t ON t.track_id = il.track_id;

CREATE VIEW album_tracks AS
    SELECT
        a.album_id
        , t.track_id
    FROM album a
    INNER JOIN track t ON t.album_id = a.album_id;
    
WITH invoice_category AS
    (
    SELECT
        invoice_id,
        CASE
            WHEN
                (
                SELECT track_id FROM invoice_tracks
                WHERE invoice_id = i.invoice_id
                EXCEPT
                SELECT track_id FROM album_tracks
                WHERE album_id = i.album_id
                ) IS NULL
        
                AND
        
                (
                SELECT track_id FROM album_tracks
                WHERE album_id = i.album_id
                EXCEPT
                SELECT track_id FROM invoice_tracks
                WHERE invoice_id = i.invoice_id
                ) IS NULL
            THEN "Album"
            ELSE "Individual Tracks"
            END AS category
    FROM invoice_tracks i
    GROUP BY invoice_id
    )
    
SELECT
    category,
    COUNT(*) AS number_of_invoices,
    CAST(COUNT(*) AS FLOAT) / (SELECT COUNT(invoice_id) FROM invoice)*100
    AS percentage_invoices
FROM invoice_category
GROUP BY category;

Done.
Done.
Done.
Done.
Done.


category,number_of_invoices,percentage_invoices
Album,114,18.566775244299674
Individual Tracks,500,81.43322475570032


Over 614 total purchases that Chinook has received over time, about the 18.6% of them are whole albums.

Therefore, I would recommend continuing to buy full albums from record companies, as we have one fifth of our revenue at stake.

### Alternative query

Below there's an alternative way to write the above query with one less intermediate subquery.

In [9]:
%%sql

DROP VIEW IF EXISTS invoice_album;

CREATE VIEW invoice_album AS
    SELECT
        il.invoice_id
        , t.album_id
    FROM invoice_line il
    INNER JOIN track t ON t.track_id = il.track_id
    GROUP BY 1;
    
WITH invoice_category AS
    (
    SELECT
        invoice_id,
        CASE
            WHEN
                (
                SELECT track_id FROM invoice_line
                WHERE invoice_id = ia.invoice_id
                EXCEPT
                SELECT track_id FROM track
                WHERE album_id = ia.album_id
                ) IS NULL
        
                AND
        
                (
                SELECT track_id FROM track
                WHERE album_id = ia.album_id
                EXCEPT
                SELECT track_id FROM invoice_line
                WHERE invoice_id = ia.invoice_id
                ) IS NULL
            THEN "Album"
            ELSE "Individual Tracks"
            END AS category
    FROM invoice_album ia
    GROUP BY invoice_id
    )
    
SELECT
    category,
    COUNT(*) AS number_of_invoices,
    CAST(COUNT(*) AS FLOAT) / (SELECT COUNT(invoice_id) FROM invoice)*100
    AS percentage_invoices
FROM invoice_category
GROUP BY category;

Done.
Done.
Done.


category,number_of_invoices,percentage_invoices
Album,114,18.566775244299674
Individual Tracks,500,81.43322475570032
